In [1]:
import pandas as pd
from google.colab import drive
get_ipython().run_line_magic('matplotlib', 'inline')
import seaborn as sns
import requests
import io
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

drive.mount('/Drive')

Drive already mounted at /Drive; to attempt to forcibly remount, call drive.mount("/Drive", force_remount=True).


In [2]:
#PARAMETERS before launching
#define if we should keep the part "X_test_challenge" from the loaded file or ignore it
Ignore_X_test_challenge = True
isUseJoblib = True
FilePath="/Drive/My Drive/"

In [3]:
# 
#Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/JulienJ-44/rakuteam/main/Features/data_features_final.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('utf-8')), index_col=0)
df['description']=df['description'].fillna("")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98728 entries, 0 to 98727
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   designation        98726 non-null  object 
 1   description        98728 non-null  object 
 2   productid          98728 non-null  int64  
 3   prdtypecode_x      84916 non-null  float64
 4   desi_nb_phrases    98728 non-null  int64  
 5   desc_nb_phrases    98728 non-null  int64  
 6   best_idf           98728 non-null  object 
 7   class_10.0         98728 non-null  float64
 8   class_40.0         98728 non-null  float64
 9   class_50.0         98728 non-null  float64
 10  class_60.0         98728 non-null  float64
 11  class_1140.0       98728 non-null  float64
 12  class_1160.0       98728 non-null  float64
 13  class_1180.0       98728 non-null  float64
 14  class_1280.0       98728 non-null  float64
 15  class_1281.0       98728 non-null  float64
 16  class_1300.0       987

In [4]:

#Drop useless columns
df = df.drop(["prdtypecode_y","productid", "designation","description", "best_idf"], axis=1)

In [5]:

df = df.replace({'prdtypecode_x': {10: 1, 2280:2,   50:3, 1280:4, 2705:5, 2522:6, 2582:7, 1560:8, 1281:9, 1920:10, 2403:11,
       1140:12, 2583:13, 1180:14, 1300:15, 2462:16, 1160:17, 2060:18,   40:19,   60:20, 1320:21, 1302:22,
       2220:23, 2905:24, 2585:25, 1940:26, 1301:0}})
df.head()

,prdtypecode_x,desi_nb_phrases,desc_nb_phrases,class_10.0,class_40.0,class_50.0,class_60.0,class_1140.0,class_1160.0,class_1180.0,class_1280.0,class_1281.0,class_1300.0,class_1301.0,class_1302.0,class_1320.0,class_1560.0,class_1920.0,class_1940.0,class_2060.0,class_2220.0,class_2280.0,class_2403.0,class_2462.0,class_2522.0,class_2582.0,class_2583.0,class_2585.0,class_2705.0,class_2905.0,desc_nb2chiffres+,desi_num,desc_num,desc_poids,desc_long,desc_vol,desc_ans_mois,desc_pieces,blanc,noir,R,G,B
0,1.0,1,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.157,0.115,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.190,0.0,0.0,0.115,0.000,0.021,0,0,0,0,0,0,0,0,0.896177,0.000007,251.471600,246.764924,246.580288
1,2.0,1,0,0.518,0.0,0.00,0.0,0.0,0.265,0.0,0.176,0.000,0.338,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,1.301,0.235,0.000,0.272,0.0,0.0,0.000,0.000,0.000,0,1,0,0,0,0,0,0,0.804087,0.000235,233.710580,232.696360,231.392644
2,3.0,1,1,0.000,0.1,0.48,0.0,0.0,0.000,0.0,0.000,0.275,0.000,0.0,0.0,0.144,0.000,0.0,0.26,0.0,0.0,0.000,0.625,0.657,0.297,0.0,0.0,0.000,0.000,0.020,1,0,0,0,0,0,0,0,0.925060,0.000000,249.340400,251.500820,253.409132
3,4.0,1,0,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.222,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0,0,0,0,0,0,0,0,0.476548,0.000221,169.924608,152.227476,149.065240
4,5.0,1,1,0.000,0.0,0.00,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.111,0.000,0,0,0,0,0,0,0,0,0.781337,0.003199,225.403116,234.072128,238.177512


In [6]:
import numpy as np

if (Ignore_X_test_challenge):
  df =df.dropna(subset=['prdtypecode_x'])
  y = df['prdtypecode_x'].values
  df =df.drop('prdtypecode_x', axis=1)
  X_train, X_test, y_train, y_test = train_test_split(
        df, y, test_size=0.2, random_state=123)
else:
  X_train = df.dropna(subset=["prdtypecode_x"], axis=0)
  y_train = X_train['prdtypecode_x'].values
  X_train =X_train.drop('prdtypecode_x', axis=1)
  X_test = df[np.isnan(df["prdtypecode_x"])]
  X_test =X_test.drop('prdtypecode_x', axis=1)
  y_test = None
  
  
print("X train:", len(X_train))
print("Y train:", len(X_train))
print("X test:", len(X_test))


X train: 67932
Y train: 67932
X test: 16984


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

X_train  = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import os
import joblib
from time import time
t0 = time()
# {}
if (isUseJoblib):
  clf1 = joblib.load(f'{FilePath}/randomforest98000.joblib')
else:
  clf1 = RandomForestClassifier(max_features ='sqrt', n_jobs = -1,n_estimators = 500)
  clf1.fit(X_train, y_train)

y_pred = clf1.predict(X_test)

print('Processing time: {} mins'.format(round((time() - t0) / 60, 2)))

if (Ignore_X_test_challenge):
  dfcross=pd.crosstab(y_test.ravel(), y_pred, rownames=['Realité'], colnames=['Prédiction'], normalize=1)
  classif = classification_report(y_test, y_pred)
  print(classif)

In [12]:
#SAVE Model
joblib.dump(clf1, f'{FilePath}/randomforest98000.joblib', compress=3)

['/Drive/My Drive//randomforest98000.joblib']

In [ ]:
import pickle as cPickle
with open(f'{FilePath}/randomforest98000.pickle', 'wb') as f:
    cPickle.dump(clf1, f)
    f.close()
#with open(f'{FilePath}/randomforest98000.pickle', 'rb') as f:
#    forest2 = cPickle.load(f)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

params_clf1 = {'n_estimators': [ 250, 500, 1000],
                'min_samples_leaf': [1, 3, 5],
                'max_features': ['sqrt', 'log2']}

gridcv = GridSearchCV(clf1, param_grid=  params_clf1, scoring= 'accuracy', cv = 3)

In [ ]:

t0 = time()

gridcv.fit(X_train, y_train)


pd.DataFrame(gridcv.cv_results_)[['params', 'mean_test_score' , 'std_test_score']]

print('Processing time: {} mins'.format(round((time() - t0) / 60, 2)))

Processing time: 96.49 mins


In [ ]:
ypred_proba = clf1.predict_proba(X_test)